In [7]:
from datetime import datetime
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import numpy as np 
import pandas as pd

Отфильтруем время на дату, день и час

In [3]:
def filter_time(datefield):
    """Столбец делится на дату, день, час и добавляются в виде отдельных столбцов"""
    i = 0
    for timestamp in datefield['datetime']:
        date_object = datetime.strptime(timestamp.split()[0], '%Y-%m-%d')
        hour = timestamp.split()[1][:2]
        # 0 - понедельник, 1 - вторник и т.д
        day = datetime.date(date_object).weekday()
        dict_year = {2011:1, 2012:2}
        year = dict_year[date_object.year]
        datefield.loc[i, 'day'] = day
        datefield.loc[i, 'hour'] = hour
        datefield.loc[i, 'year'] = year
        i += 1
    return datefield

Считаем данные и отфильтруем по времени

In [4]:
bike_dataframe = pd.read_csv("/home/emin/PycharmProjects/machine_learning/data/bike_sharing/train.csv")
bike_dataframe = filter_time(bike_dataframe)
bike_dataframe = bike_dataframe[['datetime', 'year', 'day', 'hour', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count']]
bike_dataframe.head()

,datetime,year,day,hour,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1.0,5.0,00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1.0,5.0,01,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1.0,5.0,02,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1.0,5.0,03,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1.0,5.0,04,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [5]:
column = ['year', 'day', 'hour', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed']
bike_dataframe[column][2:-1]

,year,day,hour,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
2,1.0,5.0,02,1,0,0,1,9.02,13.635,80,0.0000
3,1.0,5.0,03,1,0,0,1,9.84,14.395,75,0.0000
4,1.0,5.0,04,1,0,0,1,9.84,14.395,75,0.0000
5,1.0,5.0,05,1,0,0,2,9.84,12.880,75,6.0032
6,1.0,5.0,06,1,0,0,1,9.02,13.635,80,0.0000
...,...,...,...,...,...,...,...,...,...,...,...
10880,2.0,2.0,18,4,0,1,1,15.58,19.695,50,23.9994
10881,2.0,2.0,19,4,0,1,1,15.58,19.695,50,26.0027
10882,2.0,2.0,20,4,0,1,1,14.76,17.425,57,15.0013
10883,2.0,2.0,21,4,0,1,1,13.94,15.910,61,15.0013


In [6]:
test_dataframe = pd.read_csv("/home/emin/PycharmProjects/machine_learning/data/bike_sharing/test.csv")
test_dataframe = filter_time(test_dataframe)
test_dataframe = test_dataframe[['datetime', 'year', 'day', 'hour', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed']]
test_dataframe.head()

,datetime,year,day,hour,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1.0,3.0,00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1.0,3.0,01,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1.0,3.0,02,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1.0,3.0,03,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1.0,3.0,04,1,0,1,1,10.66,12.880,56,11.0014


Создаем случайный лес с 500 деревьями, при 3 объектах происходит расщепление и оцениваем

In [8]:
params = {'n_estimators': 1000, 'max_depth': 20, 'random_state': 0, 'min_samples_split' : 5, 'n_jobs': -1}
random_forest_object = RandomForestRegressor(**params)
# для случайных велосипедистов
collumn_fit = ['year', 'day', 'hour', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed']
# перекрестная проверка случайных велосипедистов
results = cross_val_score(random_forest_object, bike_dataframe[collumn_fit], bike_dataframe['casual'], cv=5)
print(results)

[0.56356953 0.80469413 0.2869214  0.70624468 0.82844487]


Отчёт о точности работы

In [9]:
print("Точность: %0.2f (+/- %0.2f)" % (results.mean(), results.std()*2))

Точность: 0.64 (+/- 0.40)


Обучаем случайный лес

In [10]:
casual_obj = random_forest_object.fit(bike_dataframe[collumn_fit], bike_dataframe['casual'] )
casual_obj.feature_importances_

array([0.03675721, 0.05876091, 0.35575392, 0.01658465, 0.0032051 ,
       0.14519326, 0.00929277, 0.07412475, 0.22501295, 0.05614853,
       0.01916593])

Предсказываем

In [11]:
test_dataframe['casual'] = random_forest_object.predict(test_dataframe[collumn_fit])
test_dataframe.head()

,datetime,year,day,hour,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual
0,2011-01-20 00:00:00,1.0,3.0,00,1,0,1,1,10.66,11.365,56,26.0027,2.077924
1,2011-01-20 01:00:00,1.0,3.0,01,1,0,1,1,10.66,13.635,56,0.0000,0.851279
2,2011-01-20 02:00:00,1.0,3.0,02,1,0,1,1,10.66,13.635,56,0.0000,1.267675
3,2011-01-20 03:00:00,1.0,3.0,03,1,0,1,1,10.66,12.880,56,11.0014,0.702474
4,2011-01-20 04:00:00,1.0,3.0,04,1,0,1,1,10.66,12.880,56,11.0014,0.525744


Обучаем модель на зарегистрированных велосипедистов

In [12]:
# для зарегестрированных велосипедистов
regullar_collumn = ['year', 'day', 'hour', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed']
# перекрестная проверка зарегестрированных велосипедистов
results = cross_val_score(random_forest_object, bike_dataframe[regullar_collumn], bike_dataframe['registered'], cv=5)
print(results)

[0.08239602 0.88329613 0.72592457 0.85570614 0.87275756]


Отчёт о точности работы

In [13]:
print("Точность: %0.2f (+/- %0.2f)" % (results.mean(), results.std()*2))

Точность: 0.68 (+/- 0.61)


In [14]:
registered_obj = random_forest_object.fit(bike_dataframe[regullar_collumn], bike_dataframe['registered'] )
registered_obj.feature_importances_

array([0.09291987, 0.02409135, 0.58151417, 0.05508238, 0.00104476,
       0.14248461, 0.01549591, 0.02939976, 0.02280755, 0.02698214,
       0.0081775 ])

Прогнозы на тестовом наборе данных

In [15]:
test_dataframe['registered'] = random_forest_object.predict(test_dataframe[regullar_collumn])
test_dataframe['count'] = test_dataframe['casual'] + test_dataframe['registered']
test_dataframe.head()

,datetime,year,day,hour,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-20 00:00:00,1.0,3.0,00,1,0,1,1,10.66,11.365,56,26.0027,2.077924,10.471885,12.549810
1,2011-01-20 01:00:00,1.0,3.0,01,1,0,1,1,10.66,13.635,56,0.0000,0.851279,4.966673,5.817953
2,2011-01-20 02:00:00,1.0,3.0,02,1,0,1,1,10.66,13.635,56,0.0000,1.267675,2.131863,3.399538
3,2011-01-20 03:00:00,1.0,3.0,03,1,0,1,1,10.66,12.880,56,11.0014,0.702474,1.781000,2.483474
4,2011-01-20 04:00:00,1.0,3.0,04,1,0,1,1,10.66,12.880,56,11.0014,0.525744,1.775069,2.300813


Округляем количество до целого числа

In [16]:
test_dataframe['count'] = test_dataframe['count'].apply(np.round)
test_dataframe.head()

,datetime,year,day,hour,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-20 00:00:00,1.0,3.0,00,1,0,1,1,10.66,11.365,56,26.0027,2.077924,10.471885,13.0
1,2011-01-20 01:00:00,1.0,3.0,01,1,0,1,1,10.66,13.635,56,0.0000,0.851279,4.966673,6.0
2,2011-01-20 02:00:00,1.0,3.0,02,1,0,1,1,10.66,13.635,56,0.0000,1.267675,2.131863,3.0
3,2011-01-20 03:00:00,1.0,3.0,03,1,0,1,1,10.66,12.880,56,11.0014,0.702474,1.781000,2.0
4,2011-01-20 04:00:00,1.0,3.0,04,1,0,1,1,10.66,12.880,56,11.0014,0.525744,1.775069,2.0


Вывод результата в csv таблицу

In [17]:
# генерим результаты в submission.csv
dataframe_csv = test_dataframe[['datetime', 'count']]
dataframe_csv.to_csv('/home/emin/PycharmProjects/machine_learning/data/bike_sharing/submission.csv', index=False)
dataframe_csv.head()

,datetime,count
0,2011-01-20 00:00:00,13.0
1,2011-01-20 01:00:00,6.0
2,2011-01-20 02:00:00,3.0
3,2011-01-20 03:00:00,2.0
4,2011-01-20 04:00:00,2.0


# Результаты: 0.40774